In [ ]:
import numpy as np
import random
import math

import matplotlib.pyplot as plt

from math import exp
from sklearn.model_selection import train_test_split

In [3]:
class logistic_regression:

    def __init__(self, num_epochs, train_data, test_data, num_features, learn_rate, activation):
        self.train_data = train_data
        self.test_data = test_data
        self.num_features = num_features
        self.num_outputs = self.train_data.shape[1] - num_features
        self.use_sigmoid = activation
        self.w = np.random.uniform(-0.5, 0.5, num_features)
        self.b = np.random.uniform(-0.5, 0.5, self.num_outputs)
        self.learn_rate = learn_rate
        self.num_epochs = num_epochs
        self.gradient = np.zeros(self.num_outputs)

    def logisticactivtion(self, z_vec):
        if self.use_sigmoid == True:
            y = 1/(1 + np.exp(z_vec))
        else:
            y = z_vec
        return y

    def rmse(self, output, actual):
        squared_mean = np.square(output - actual)//output.shape[0]
        sum_sq_mean = np.sum(squared_mean)
        return np.sqrt(sum_sq_mean)

    def output(self, x_vec):
        z = x_vec.dot(self.w)-self.b
        output = self.logisticactivtion(z)
        return output

    def gradient(self, output, actual): #derivative of output w.r.t x
        if self.use_sigmoid == True:
            gradient = (output - actual) * output * (1 - output) # there are only two discrete outcomes (1 or 0), this is a Bernoulli distribution
        else: #if not use sigmoid then linear
            gradient = output - actual
        return gradient

    def update(self, gradient, x_vec):
        self.w += self.learn_rate * gradient * x_vec
        self.b += self.learn_rate * gradient * 1

    def encoder(self, w): #get parameters and put them in fixed-shape model
        self.w = w[0:self.num_features]
        self.b = w[self.num_features:] #get b as the rest of parameters

    def predicted_output(self, data, w):
        self.encoder(w)
        fx = np.zeros(data.shape[0])
        for i in range(0, data.shape[0]):
            input = data[i, 0:self.num_features] #i - which rows to get and 0:self.num_features - which columns to get
            predicted_output = self.output(input)
            fx[i] = predicted_output
        return fx


In [7]:
class MCMC:
    
    def __init__(self,topology):
        self.topology = topology #[Weights, Bias]

    def rmse(self, output, actual):
        squared_mean = np.square(output - actual)//output.shape[0]
        sum_sq_mean = np.sum(squared_mean)
        return np.sqrt(sum_sq_mean)

    def likelihood_function(self, model, data, tausq, w):
        #tausq is the extent of variation among the effects observed in different studies 
        y = data[:, self.topology[0]] #collect all columns of weights that act as inputs
        fx = model.predicted_output(data, w)
        accuracy = self.rmse(fx, y)
        loss = (-1/2) * np.log(2 * math.pi * tausq) - 1/2 * np.square(y - fx) / tausq
            
        return [np.sum(loss), fx, accuracy]

    def prior_normal(self, w, sigmasq, nu_1, nu_2, tausq):
        num_w = self.topology[0] + 1 #number of weights
        #find log of the PDF (or maximum likelihood estimation) of normal distribution, our prior distribution
        log_mle = (-1/2 * num_w * np.log(sigmasq)) - (1/2 * np.sum(np.square(w)) / sigmasq) - ((1 + nu_1) * np.log(tausq)) - (nu_2 / tausq)
        return log_mle

    def sampler(self):
        samples = self.samples
        ##split data into train and test set
        data_train, data_test = train_test_split(self.data, test_size=0.25)
        train_size = data_train.shape[0]
        test_size = data_test.shape[0]

        ##Initialize train and test set
        X_train = np.linspace(0, 1, train_size)
        X_test = np.linspace(0, 1, test_size)

        y_train = data_train[:, self.topology[0]]
        y_test = data_test[:, self.topology[0]]

        #number of weights and bias
        w_size = self.topology[1] + self.topology[0]

        ##Initialize fx of train and test data over all samples
        fx_train = np.ones((samples, train_size)) #array with row-columns as samples-train_size
        fx_test = np.ones((samples, test_size))
        rmse_train = np.zeros(samples)
        rmse_test = np.zeros(samples)

        ##Initialize posterior distribution of weights over all samples 
        pos_w = np.ones((samples, w_size)) # values of all weights and bias drawn from posterior distribution over all samples
        pos_tau = np.ones((samples, 1))

        ##Propose initial distribution for w
        w = np.random.randn(w_size) #return a sample of w from standard normal distribution 
        proposed_w = np.random.randn(w_size) #proposed value of w
        step_w = 0.02  # defines how much variation you need in changes to w
        step_eta = 0.01 # eta is an additional parameter to cater for noise in predictions (Gaussian likelihood)

        ##Evaluate initial w (at epoch 0)
        #Model
        model = logistic_regression(0, data_train, data_test, self.topology[0], 0.1, self.regression)

        predicted_train_output = model.predicted_output(data_train, w)
        predicted_test_output = model.predicted_output(data_test, w)
        eta = np.log(np.var(y_train - predicted_train_output))
        proposed_tau = np.exp(eta)

        ##Prior Distribution
        #sigmasq_nu = {'sigmasq': 5, 'nu_1': 0, 'nu_2':0} #Create a dict to avoid the problem positional argument (proposed_tau) can't appear after keyword arg (sigmasq=5,..)
        sigmasq = 5
        nu_1 = 0
        nu_2 = 0
        prior = self.prior_normal(w, sigmasq, nu_1, nu_2, proposed_tau)

        ##Likelihood
        [likelihood, fx_train, rmsetrain] = self.likelihood_function(model, data_train, proposed_tau, w)
        print('Initial Likelihood is:', likelihood)

        ##Propose the moves
        naccept = 0
        for i in range(samples-1):
            proposed_w = w + np.random.normal(0, step_w, w_size)
            proposed_eta = eta + np.random.normal(0, step_eta, 1)
            proposed_tau = np.exp(proposed_eta)

            #Find likelihood and prior for proposed w
            prior_proposal = self.prior_normal(proposed_w, sigmasq, nu_1, nu_2, proposed_tau)
            [likelihood_proposal, fx_train_propose, rmsetrain_propose] = self.likelihood_function(model, data_train, proposed_tau, proposed_w)

            #Find the probability of transitioning from state w to proposed w, which has a unique stationary distribution π(w)
            prior_diff = prior_proposal - prior
            likelihood_diff = likelihood_proposal - likelihood

            accept_prob = np.min(1, np.exp(prior_diff + likelihood_diff)) #because both values in log form
            #Generate a random uniform u
            u = random.uniform(0, 1)

            if u < accept_prob or u == accept_prob:
                naccept += 1
                likelihood = likelihood_proposal
                prior = prior_proposal
                eta = proposed_eta
                w = proposed_w

                print(likelihood, prior, w, rmse_train, 'accepted')

                #Update posterior distribution
                pos_w[i+1, ] = proposed_w #means there is only 1 column or 1-D arra
                pos_tau[i+1, ] = proposed_tau
                fx_train[i+1, ] = fx_train_propose
                rmse_train[i+1, ] = rmsetrain_propose

            else: #Keep posterior distribution (!!!!) intact
                pos_w[i+1, ] = pos_w[i, ]
                pos_tau[i+1, ] = pos_tau[i, ]
                fx_train[i+1, ] = fx_train[i, ]
                rmse_train[i+1, ] = rmse_train[i, ]

        accept_ratio = 100 * (naccept / samples)
        print(accept_ratio, '% was accepted')

        ##Use burnin
        #Start at x, then run the Markov chain for n steps, from which throw away all the data (no output). n is the burn-in period. After the burn-in run normally, using each iterate in MCMC calculations
        burnin = 0.25 * samples  # use post burn in samples

        pos_w = pos_w[int(burnin):, ]
        pos_tau = pos_tau[int(burnin):, ] 
        fx_train = fx_train[int(burnin):, ]
        rmse_train = rmse_train[int(burnin):]

        ##Test Bayesian model via posterior distributions of a certain # of samples over n trials
        num_trials = 100

        accuracy_posterior =np.zeros(num_trials)
        for i in range(num_trials):
            w_drawn = np.random.normal(pos_w.mean(axis=0), pos_w.std(axis=0), w_size) #draw random samples of w from normal distribution with parameters: mean and std of each value of vector 'posterior w' over the certain # of samples
            #axis=0 means calculated mean across the rows
            tau_drawn = np.random.normal(pos_tau.mean, pos_tau.std)
            #Check loss and accuracy of predictions made with posterior weights    
            [loss_posterior, fx_posterior, accuracy_posterior[i]] = self.likelihood_function(model, data_train, tau_drawn, w_drawn)
            print('Testing posterior distributions:', i, loss_posterior, accuracy_posterior[i], tau_drawn, pos_tau.mean(), pos_tau.std())
            print('Mean and std of accuracy of posterior test:', accuracy_posterior.mean(), accuracy_posterior.std())

        return (pos_w, pos_tau, fx_train, rmse_train, accept_ratio)